In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Start Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("AmazonVineReview").getOrCreate()

In [41]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Digital_Ebook_Purchase_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)

# Show DataFrame
# clean and reassign then add again
df = df.dropna()
df.show()
print(df.count())

+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|    product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+--------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   33605939| RGYFDX8QXKEIR|B007KO2MLO|     328837464|           Big Maria|Digital_Ebook_Pur...|          4|            0|          0|   N|                N|              Quirky|Elmore Leonard me...| 2013-09-09|
|         US|   34058393|R13CBGTMNV9R8Z|B005FLODDE|     764276359|The Woman Who Was...|Digital_Ebook_Pur

In [42]:
print(df.count())

5101476


In [15]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()


+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RGYFDX8QXKEIR|          4|            0|          0|   N|                N|
|R13CBGTMNV9R8Z|          4|            1|          2|   N|                Y|
| R7DRFHC0F71O0|          5|            0|          0|   N|                N|
|R27LUKEXU3KBXQ|          5|            1|          1|   N|                Y|
|R1VXTPUYMNU687|          5|            1|          2|   N|                N|
|R30DKW1GJWLPZC|          3|            1|          2|   N|                Y|
|R18DPFG2FALJI9|          5|            0|          0|   N|                Y|
|R24D677N5WBW5Q|          5|            0|          0|   N|                Y|
|R2FCJ9BQLSIOR3|          5|            0|          0|   N|                Y|
|R1R6K4MAKDWTXI|          4|            0|          0|   N|     

In [23]:
# Filter vine_df for total_votes greater than 20
from pyspark.sql.functions import col

vine_filter1_df = vine_df.filter(col("total_votes") >= 20)
vine_filter1_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| R4IAIDV5EE84W|          1|            8|         21|   N|                Y|
|R26MB2DA7ROT4P|          1|            2|         21|   N|                Y|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
| RYBZ6CTBOGI0H|          1|            3|         84|   N|     

In [18]:
# Filter for helpful vote percentage

vine_filter2_df = vine_filter1_df.filter((col("helpful_votes")/col("total_votes")) >= 0.5)
vine_filter2_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [26]:
# Filter to retrieve all the vine yes

vine_filteryes_df = vine_filter2_df.filter(col("vine") == "Y")
vine_filteryes_df.show()

+---------+-----------+-------------+-----------+----+-----------------+
|review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+---------+-----------+-------------+-----------+----+-----------------+
+---------+-----------+-------------+-----------+----+-----------------+



In [24]:
# Filter to retrieve all the vine no

vine_filterno_df = vine_filter2_df.filter(col("vine") == "N")
vine_filterno_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R39ESX43X1SA5T|          1|           36|         47|   N|                Y|
|R31QRZ3JAMPBB4|          1|           16|         26|   N|                Y|
|R23FKBEXURC3SN|          5|           28|         36|   N|                N|
| RTRVYV0GWUTCK|          5|           18|         25|   N|                Y|
|R3QLZAI37SNOOC|          2|           30|         31|   N|                Y|
|R1QYZU3RS4CY4F|          5|           66|         76|   N|                N|
|R2UR5OLN1BU87J|          2|           21|         23|   N|                Y|
|R38NW44NY19VAZ|          5|           30|         32|   N|                Y|
| RLUFKZ223OR3L|          2|           20|         23|   N|                N|
|R3H9D0U7BCE6FU|          4|           23|         24|   N|     

In [44]:
# Percentage of 5 star reviews for entire data set for comparison
total_review_all = df.count()
total_fivestar = (df.filter(col("star_rating") == 5)).count()
print(total_fivestar/total_review_all*100)

57.87542664123089


In [32]:
# Total number of reviews
total_review_count = vine_filter2_df.count()
print(total_review_count)

65145


In [33]:
# Total number of 5 start reviews for vine reviews
vine_5star_df = vine_filter2_df.filter(col("star_rating") == 5)
fivestar_count = vine_5star_df.count()
percent_fivestar = fivestar_count/total_review_count*100
print(percent_fivestar)

37.87243840663136


In [43]:
# Percentage of 5-star reviews for paid (Y) vine reviews
total_yes = vine_filteryes_df.count()
yes_fivestar = (vine_filteryes_df.filter(col("star_rating") == 5)).count()
try:
  percent_yes = yes_fivestar/total_yes*100
  print(percent_yes)
except ZeroDivisionError:
  print("There are no paid vine reviews.")

There are no paid vine reviews.


In [39]:
# Percentage of 5-star reviews for unpaid (N) vine reviews
total_no = vine_filterno_df.count()
no_fivestar = (vine_filterno_df.filter(col("star_rating") == 5)).count()
try:
  percent_no = no_fivestar/total_no*100
  print(percent_no)
except ZeroDivisionError:
  print("There are no unpaid vine reviews.")


37.87243840663136


In [5]:
# Read data in from an S3 link

# template_url = "https://<bucket-name>.s3.amazonaws.com/<folder-name>/<file-name>"

# example_url = "https://dataviz-curriculum.s3.amazonaws.com/data-folder/data.csv"
